In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
layer = keras.layers.Dense(3)
layer.build((None, 4))

print('weights: ', len(layer.weights))
print('trainable weights: ', len(layer.trainable_weights))
print('non trainable weights: ', len(layer.non_trainable_weights))

In [ ]:
layer = keras.layers.Dense(3)
layer.build((None, 4))
layer.trainable = False

print('weights: ', len(layer.weights))
print('trainable weights: ', len(layer.trainable_weights))
print('non trainable weights: ', len(layer.non_trainable_weights))

In [ ]:
layer1 = keras.layers.Dense(3, activation='relu')
layer2 = keras.layers.Dense(3, activation='sigmoid')
model = keras.Sequential([keras.Input(shape=(3, )), layer1, layer2])

layer1.trainable = False
initial_layer1_weights_values = layer1.get_weights()

model.compile(optimizer='adam', loss='mse')
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

final_layer1_weights_values = layer1.get_weights()

np.testing.assert_allclose(
    initial_layer1_weights_values[0], final_layer1_weights_values[0]
)

np.testing.assert_allclose(
    initial_layer1_weights_values[1], final_layer1_weights_values[1]
)

In [ ]:
inner_model = keras.Sequential([
    keras.Input(shape=(3, )),
    keras.layers.Dense(3, activation='relu'),
    keras.layers.Dense(3, activation='relu'),
])

model = keras.Sequential([
    keras.Input(shape=(3, )),
    inner_model,
    keras.layers.Dense(3, activation='sigmoid')
])

model.trainable = False
print(inner_model.trainable)
print(inner_model.layers[0].trainable)

In [ ]:
base_model = keras.applications.Xception(weights='imagenet', input_shape=(150, 150, 3), include_top=False)

In [ ]:
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1)(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

# model.fit(new_dataset, epochs=20, callbacks=..., validation_data=...)

In [ ]:
base_model.trainable = True

model.compile(optimizer=keras.optimizers.Adam(1e-5), loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

# model.fit(new_dataset, epochs=10, callbacks=..., validation_data=...)

In [ ]:
base_model = keras.applications.Xception(weights='imagenet', input_shape=(150, 150, 3), include_top=False)
base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()

for inputs, targets in new_dataset:
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss_value = loss_fn(targets, predictions)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
tfds.disable_progress_bar()

train_ds, validation_ds, test_ds = tfds.load('cats_vs_dogs', split=['train[:40%]', 'train[40%:50%]', 'train[50%:60%]'], as_supervised=True)

print('Number of training samples: %d' % (tf.data.experimental.cardinality(train_ds)))
print('Number of validation samples: %d' % (tf.data.experimental.cardinality(validation_ds)))
print('Number of testing samples: %d' % (tf.data.experimental.cardinality(test_ds)))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(train_ds.take(9)):
    ax = plt.subplot(3, 3, i+1)
    plt.imshow(image)
    plt.title(int(label))
    plt.axis('off')

In [ ]:
size = (150, 150)

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

In [ ]:
batch_size = 32

train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal'),
    layers.experimental.preprocessing.RandomRotation(0.1)
])

In [ ]:
import numpy as np

for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    first_image = images[0]

    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0), training=True)
        plt.imshow(augmented_image[0].numpy().astype('int32'))
        plt.title(int(labels[0]))
        plt.axis('off')

In [ ]:
base_model = keras.applications.Xception(weights='imagenet', input_shape=(150, 150, 3), include_top=False)
base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)

norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2

x = norm_layer(x)
norm_layer.set_weights([mean, var])

x = base_model(x, training=False)
x = keras.layers.GlobalMaxPooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1)(x)

model = keras.Model(inputs, outputs)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

history = model.fit(train_ds, epochs=20, validation_data=validation_ds)

In [ ]:
base_model.trainable = True
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(1e-5), loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])
model.fit(train_ds, epochs=10, validation_data=validation_ds)